Librerias

In [2]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import point, polygon
import rasterio
import rasterstats
import pandas as pd
from rasterstats import zonal_stats


Carga de datos



Limites administrativos


In [3]:
path = "../Data/Raw/comunas.shp"
gdf_regional = gpd.read_file(path)


Aeropuertos

In [4]:
path1 = "../Data/Raw/Aeropuertos.shp"
gdf_aeropuertos = gpd.read_file(path1)


Raster


In [5]:

ruta_raster = ("../Data/Raw/VNL_npp_2024_global_vcmslcfg_v2_c202502261200.average_masked.dat.tif")

Datos Censo

In [6]:
ruta1 = "../Data/Raw/D1_Poblacion-censada-por-sexo-y-edad-en-grupos-quinquenales.xlsx"
df_poblacion = pd.read_excel(ruta1, sheet_name = "2", skiprows=3)

In [7]:
df_poblacion.head(2)

,Código región,Región,Código provincia,Provincia,Código comuna,Comuna,Población censada,Hombres,Mujeres,Razón hombre-mujer
0,0,País,0.0,País,0.0,País,18480432.0,8967033.0,9513399.0,94.3
1,15,Arica y Parinacota,151.0,Arica,15101.0,Arica,241653.0,118844.0,122809.0,96.8


#### Procesamiento de datos

Armonizacion de CRS

In [8]:
gdf_regional = gdf_regional.to_crs("EPSG:4326")

In [9]:
gdf_aeropuertos = gdf_aeropuertos.to_crs("EPSG:4326")

Procesamiento de aeropuertos

In [10]:
gdf_aeropuertos = gdf_aeropuertos[gdf_aeropuertos['categoría'] != 'Aeródromo']

Procesamiento de las columnas de region 

In [11]:
gdf_regional.head(2)

,objectid,shape_leng,dis_elec,cir_sena,cod_comuna,codregion,st_area_sh,st_length_,Region,Comuna,Provincia,geometry
0,48,170038.624165,16,8,6204,6,9.685774e+08,206184.271675,Región del Libertador Bernardo O'Higgins,Marchigüe,Cardenal Caro,"POLYGON ((-71.80071 -34.20728, -71.79992 -34.2..."
1,29,125730.104795,15,8,6102,6,4.157446e+08,151911.576827,Región del Libertador Bernardo O'Higgins,Codegua,Cachapoal,"POLYGON ((-70.35293 -33.95263, -70.35243 -33.9..."


In [12]:
gdf_regional.Region.unique()

array(["Región del Libertador Bernardo O'Higgins",
       'Región de La Araucanía', 'Región Metropolitana de Santiago',
       'Región de Los Lagos', 'Región de Los Ríos', 'Región del Maule',
       'Región de Coquimbo', 'Región de Magallanes y Antártica Chilena',
       'Zona sin demarcar', 'Región de Valparaíso', 'Región del Bío-Bío',
       'Región de Aysén del Gral.Ibañez del Campo', 'Región de Tarapacá',
       'Región de Arica y Parinacota', 'Región de Antofagasta',
       'Región de Ñuble', 'Región de Atacama'], dtype=object)

In [13]:
df_poblacion.head(2)

,Código región,Región,Código provincia,Provincia,Código comuna,Comuna,Población censada,Hombres,Mujeres,Razón hombre-mujer
0,0,País,0.0,País,0.0,País,18480432.0,8967033.0,9513399.0,94.3
1,15,Arica y Parinacota,151.0,Arica,15101.0,Arica,241653.0,118844.0,122809.0,96.8


In [14]:
df_poblacion.Región.unique()

array(['País', 'Arica y Parinacota', 'Tarapacá', 'Antofagasta', 'Atacama',
       'Coquimbo', 'Valparaíso', 'Metropolitana de Santiago',
       "Libertador General Bernardo O'Higgins", 'Maule', 'Ñuble',
       'Biobío', 'La Araucanía', 'Los Ríos', 'Los Lagos',
       'Aysén del General Carlos Ibáñez del Campo',
       'Magallanes y de la Antártica Chilena', nan], dtype=object)

In [15]:
diccionario_regiones = {
    "Arica y Parinacota": "Región de Arica y Parinacota",
    "Tarapacá": "Región de Tarapacá",
    "Antofagasta": "Región de Antofagasta",
    "Atacama": "Región de Atacama",
    "Coquimbo": "Región de Coquimbo",
    "Valparaíso": "Región de Valparaíso",
    "Metropolitana de Santiago": "Región Metropolitana de Santiago",
    "Libertador General Bernardo O'Higgins": "Región del Libertador Bernardo O'Higgins",
    "Maule": "Región del Maule",
    "Ñuble": "Región de Ñuble",
    "Biobío": "Región del Bío-Bío",
    "La Araucanía": "Región de La Araucanía",
    "Los Ríos": "Región de Los Ríos",
    "Los Lagos": "Región de Los Lagos",
    "Aysén del General Carlos Ibáñez del Campo": "Región de Aysén del Gral.Ibañez del Campo",
    "Magallanes y de la Antártica Chilena": "Región de Magallanes y Antártica Chilena",
    "País": "Zona sin demarcar",
    np.nan: "Zona sin demarcar"    
}


In [16]:
df_poblacion = df_poblacion.replace({"Región":diccionario_regiones})

In [17]:
df_poblacion

,Código región,Región,Código provincia,Provincia,Código comuna,Comuna,Población censada,Hombres,Mujeres,Razón hombre-mujer
0,0,Zona sin demarcar,0.0,País,0.0,País,18480432.0,8967033.0,9513399.0,94.3
1,15,Región de Arica y Parinacota,151.0,Arica,15101.0,Arica,241653.0,118844.0,122809.0,96.8
2,15,Región de Arica y Parinacota,151.0,Arica,15102.0,Camarones,861.0,430.0,431.0,99.8
3,15,Región de Arica y Parinacota,152.0,Parinacota,15201.0,Putre,1547.0,846.0,701.0,120.7
4,15,Región de Arica y Parinacota,152.0,Parinacota,15202.0,General Lagos,508.0,261.0,247.0,105.7
...,...,...,...,...,...,...,...,...,...,...
344,12,Región de Magallanes y Antártica Chilena,123.0,Tierra del Fuego,12303.0,Timaukel,157.0,95.0,62.0,153.2
345,12,Región de Magallanes y Antártica Chilena,124.0,Última Esperanza,12401.0,Natales,24152.0,12064.0,12088.0,99.8
346,12,Región de Magallanes y Antártica Chilena,124.0,Última Esperanza,12402.0,Torres del Paine,203.0,118.0,85.0,138.8
347,NaN,Zona sin demarcar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
gdf_regional = gdf_regional[gdf_regional['Region'] != 'Zona sin demarcar']

In [19]:
df_poblacion = df_poblacion[df_poblacion['Región'] != 'Zona sin demarcar']

In [20]:
df_poblacion.Región.unique()

array(['Región de Arica y Parinacota', 'Región de Tarapacá',
       'Región de Antofagasta', 'Región de Atacama', 'Región de Coquimbo',
       'Región de Valparaíso', 'Región Metropolitana de Santiago',
       "Región del Libertador Bernardo O'Higgins", 'Región del Maule',
       'Región de Ñuble', 'Región del Bío-Bío', 'Región de La Araucanía',
       'Región de Los Ríos', 'Región de Los Lagos',
       'Región de Aysén del Gral.Ibañez del Campo',
       'Región de Magallanes y Antártica Chilena'], dtype=object)

In [21]:
gdf_regional.Region.unique()

array(["Región del Libertador Bernardo O'Higgins",
       'Región de La Araucanía', 'Región Metropolitana de Santiago',
       'Región de Los Lagos', 'Región de Los Ríos', 'Región del Maule',
       'Región de Coquimbo', 'Región de Magallanes y Antártica Chilena',
       'Región de Valparaíso', 'Región del Bío-Bío',
       'Región de Aysén del Gral.Ibañez del Campo', 'Región de Tarapacá',
       'Región de Arica y Parinacota', 'Región de Antofagasta',
       'Región de Ñuble', 'Región de Atacama'], dtype=object)

Filtrar columnas relevantes

In [22]:
gdf_regional =gdf_regional[['Region','Comuna','geometry']]
df_poblacion =df_poblacion[['Región','Comuna','Población censada']]

In [23]:
gdf_regional.head(2)

,Region,Comuna,geometry
0,Región del Libertador Bernardo O'Higgins,Marchigüe,"POLYGON ((-71.80071 -34.20728, -71.79992 -34.2..."
1,Región del Libertador Bernardo O'Higgins,Codegua,"POLYGON ((-70.35293 -33.95263, -70.35243 -33.9..."


In [24]:
df_poblacion.head(2)

,Región,Comuna,Población censada
1,Región de Arica y Parinacota,Arica,241653.0
2,Región de Arica y Parinacota,Camarones,861.0


Procesamiento de comuna

In [25]:
gdf_regional.shape

(345, 3)

In [26]:
df_poblacion.shape

(346, 3)

Comparacion de Arrays numpy

In [27]:
arr1 = np.setdiff1d(gdf_regional.Comuna.unique(), df_poblacion.Comuna.unique())
arr2 = np.setdiff1d(df_poblacion.Comuna.unique(), gdf_regional.Comuna.unique())

In [28]:
arr1

array(['Los Alamos', 'Los Angeles', 'Marchigüe', 'Paihuano', 'Ranquil',
       'San Pedro de la Paz', 'Treguaco'], dtype=object)

In [29]:
arr2

array(['Antártica', 'Los Álamos', 'Los Ángeles', 'Marchihue', 'Paiguano',
       'Ránquil', 'San Pedro de La paz', 'Trehuaco'], dtype=object)

Eliminamos la Antartica

In [30]:
df_poblacion =  df_poblacion[df_poblacion['Comuna'] != 'Antártica']

Solucionar los nombres de las comunas

In [31]:
diccionario_comunas = {
    'Los Álamos': 'Los Alamos',
    'Los Ángeles': 'Los Angeles',
    'Marchihue': 'Marchigüe',
    'Paiguano': 'Paihuano',
    'Ránquil': 'Ranquil',
    'San Pedro de La paz': 'San Pedro de la Paz',
    'Trehuaco': 'Treguaco'
}

In [32]:
df_poblacion = df_poblacion.replace({"Comuna": diccionario_comunas})

In [33]:
df_poblacion

,Región,Comuna,Población censada
1,Región de Arica y Parinacota,Arica,241653.0
2,Región de Arica y Parinacota,Camarones,861.0
3,Región de Arica y Parinacota,Putre,1547.0
4,Región de Arica y Parinacota,General Lagos,508.0
5,Región de Tarapacá,Iquique,199587.0
...,...,...,...
342,Región de Magallanes y Antártica Chilena,Porvenir,6809.0
343,Región de Magallanes y Antártica Chilena,Primavera,431.0
344,Región de Magallanes y Antártica Chilena,Timaukel,157.0
345,Región de Magallanes y Antártica Chilena,Natales,24152.0


Geodataframe poblacional a partir de un merge

In [34]:
gdf_poblacion = gdf_regional.merge(df_poblacion, left_on = 'Comuna', right_on= 'Comuna', how= 'left').drop(columns= ['Región'])

In [35]:
gdf_poblacion

,Region,Comuna,geometry,Población censada
0,Región del Libertador Bernardo O'Higgins,Marchigüe,"POLYGON ((-71.80071 -34.20728, -71.79992 -34.2...",8715.0
1,Región del Libertador Bernardo O'Higgins,Codegua,"POLYGON ((-70.35293 -33.95263, -70.35243 -33.9...",13955.0
2,Región del Libertador Bernardo O'Higgins,Coinco,"POLYGON ((-70.90058 -34.23703, -70.90063 -34.2...",7916.0
3,Región del Libertador Bernardo O'Higgins,Coltauco,"POLYGON ((-71.02492 -34.17053, -71.0232 -34.17...",21373.0
4,Región de La Araucanía,Cholchol,"POLYGON ((-72.95898 -38.46877, -72.95797 -38.4...",13167.0
...,...,...,...,...
340,Región de Ñuble,San Nicolás,"POLYGON ((-72.24207 -36.35227, -72.24014 -36.3...",15099.0
341,Región de Ñuble,San Fabián,"POLYGON ((-71.24834 -36.45596, -71.24827 -36.4...",5245.0
342,Región de Ñuble,Ñiquén,"POLYGON ((-72.09907 -36.13712, -72.09819 -36.1...",12797.0
343,Región de Ñuble,El Carmen,"POLYGON ((-71.74229 -36.82882, -71.74224 -36.8...",13186.0


Manejo del raster

In [36]:
brillo_promedio = zonal_stats(gdf_poblacion, ruta_raster, stats = ['mean'])

c:\Users\marcel gonzalez\OneDrive\Desktop\Analisis de datos 2025\.venv\Lib\site-packages\rasterstats\io.py:335: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


In [37]:
brillo_total = zonal_stats(gdf_poblacion, ruta_raster, stats = ['sum'])

In [38]:
gdf_poblacion['brillo_promedio'] = [stat['mean'] for stat in brillo_promedio]

In [39]:
gdf_poblacion['brillo_total'] = [stat['sum'] for stat in brillo_total]

In [40]:
gdf_poblacion

,Region,Comuna,geometry,Población censada,brillo_promedio,brillo_total
0,Región del Libertador Bernardo O'Higgins,Marchigüe,"POLYGON ((-71.80071 -34.20728, -71.79992 -34.2...",8715.0,0.334509,1242.367920
1,Región del Libertador Bernardo O'Higgins,Codegua,"POLYGON ((-70.35293 -33.95263, -70.35243 -33.9...",13955.0,1.253766,2009.787598
2,Región del Libertador Bernardo O'Higgins,Coinco,"POLYGON ((-70.90058 -34.23703, -70.90063 -34.2...",7916.0,1.886844,1043.424805
3,Región del Libertador Bernardo O'Higgins,Coltauco,"POLYGON ((-71.02492 -34.17053, -71.0232 -34.17...",21373.0,1.158396,1446.837036
4,Región de La Araucanía,Cholchol,"POLYGON ((-72.95898 -38.46877, -72.95797 -38.4...",13167.0,0.102332,259.411346
...,...,...,...,...,...,...
340,Región de Ñuble,San Nicolás,"POLYGON ((-72.24207 -36.35227, -72.24014 -36.3...",15099.0,0.195490,638.079346
341,Región de Ñuble,San Fabián,"POLYGON ((-71.24834 -36.45596, -71.24827 -36.4...",5245.0,0.038722,346.060638
342,Región de Ñuble,Ñiquén,"POLYGON ((-72.09907 -36.13712, -72.09819 -36.1...",12797.0,0.144303,413.716370
343,Región de Ñuble,El Carmen,"POLYGON ((-71.74229 -36.82882, -71.74224 -36.8...",13186.0,0.205455,797.781433
